# Pytorch tutorial

## 5. OPTIMIZING MODEL PARAMETERS
https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

모델과 데이터가 있다 할때, parameters를 사용자가 가진 data에 맞게 optimize를 하여 모델을 train, validate, test하고자 한다.  
model training은 반복적인 process이다. 각 iteration에서 모델은 output을 계산하고, output으로부터 loss(error)를 측정하여, parameters에 대한 loss의 미분을 collect한다. 그 후, 이러한 parameters를 gradient descent를 통해 optimize한다.  
이 process의 더 구체적인 내용으로는  [backpropagation from 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8) 를 참고하라고 나와있다.

`-` 시작전 준비 코드

In [3]:
import torch
from torch import nn # Model build에 필요한 building block을 제공해주는 라이브러리
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data", # root directory
    train = True, # 
    download = True, # 인터넷에서 다운로드 받아 root directory에 저장
    transform=ToTensor() # 데아터를 tensor로 변환
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download = True,
    transform = ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
        
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hyperparameters

Hyperparameters는 사용자가 모델 optimization 과정을 조절하도록 해주는 parameters이다. hyperparameters가 달라짐에 따라 model training과 convergence rates에 영향을 줄 수 있다.
- Hyperparameter tuning 관련 글: https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html

여기서, 다음의 hyperparameters를 training을 위해 설정한다:
- Epochs의 수: Datset에 대해 iterate를 할 횟수
- Batch size: Parameters 업데이트 전에 network를 propagate하게 될 데이터 samples의 개수
- Learning rate(LR): 힌번의 epoch(혹은 batch)마다 모델 parameters를 얼마나 업데이트 할지, 작은 LR값은 learning speed가 느리다. 반면에 큰 값은 training동안 예상치못한 결과를 낼수도 있다. 


In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop

사용자가 hyperparameters를 설정하면, 다음으로 optimization loop를 통해 model을 training하고 optimization 할 수 있다.  
optimization loop의 각각 iteration은 epoch 한번이라 불린다.  
각 Epoch은 2개의 부분으로 이루어져있다.
- The Train Loop - training dataset에 대해서 iterate를 하고 training data에 대한 최적의 parameters로 수렴하려 한다.
- The Validation/Test Loop: valid/test dataset에 대해서 iterate를 하며, 모델 성능이 향상되고 있는지 확인하기 위해 사용된다.


## Loss Function

training data가 제시되었을 때, untrained된 network는 정확한 답을 주지 않을 것이다.  
이 때 Loss function이 실제 답과 model이 예측한 값 사이의 틀린 정도를 측정해준다. 그리고 model training에서 이 loss function 값을 minimize 하는 것이 목표이다.  
loss를 계산하기 위해, 사용자는 input을 모델에 입력해서 prediction을 만들고 이를 실제 data label과 비교함으로써 얻을 수 있다.

흔히 사용되는 loss function의 예시는 다음과 같다. 
- regression task에 쓰이는 `nn.MSELoss`(Mean Squre Error) 
- classification에 쓰이는 `nn.NLLLoss` (Negative Log Likelihood)
- `nn.Softmax`, `nn.NLLLoss`를 합친 `nn.CrosEntropyLoss`

여기서는 logis를 normalize하고 prediction error를 측정하는 `nn.CrossEntropyLoss`에 model output을 입력한다.
- logits는 0과 1사이의 값으로 변환된 값이라고 보면 될 것 같다.


In [5]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

## Optimizer

Optimization은 model parameters를 조정하여 training step에서 model error를 줄이기위한 과정이다. `Optimization algorithm`은 어떻게 이러한 과정이 수행되는지 정의한다(이번 예시에서는 Stochastic Gradient Descent를 사용한다). 모든 optimization logic은 `otimizer` object에 담겨있다. optimizer는 SGD optimizer말고도 ADAM, RMSProp와 같은 다양한 optimizer가 pytorch에 있다.  
사용자는 먼저 training 되어야할 model의 parameters를 optimizer에 registering함으로써 optimizer를 initialize한다. 그리고 learning rate역시 optimizer에 건내준다.

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

training loop 내부에서 optimization은 다음 3가지 steps으로 이루어진다:
- `optimizer.zero_grad()`를 호출해서 model parameters의 gradient를 reset한다. Gradients는 default로 더해지면서 쌓이게 되는데, 이걸 방지하기 위해 사용자는 각 iteration마다 zero로 만드는 것을 명시한다.
- `loss.backward()`를 호출하여 prediction loss를 Backpropagate한다. Pytorch는 각 parameters의 loss에 대한 gradient를 축적한다.
- Gradients를 계산한 다음, `optimizer.step()`를 호출하여 backward에서 계산된 gradients를 기반으로 parameters를 수정한다.

## 전체 구현 코드

train_loop를 정의하여 optimization loop를 구현하고, test_loop로 test data에 대한 model performance를 측정한다.

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        
        pred = model(X)
        loss = loss_fn(pred,y)
        
        optimizer.zero_grad() # backward에 문제가 없도록 먼저 zero_grad를 실행
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.303218  [   64/60000]
loss: 2.291858  [ 6464/60000]
loss: 2.273129  [12864/60000]
loss: 2.263830  [19264/60000]
loss: 2.244684  [25664/60000]
loss: 2.212411  [32064/60000]
loss: 2.225910  [38464/60000]
loss: 2.188714  [44864/60000]
loss: 2.175487  [51264/60000]
loss: 2.147411  [57664/60000]
Test Error: 
 Accuracy: 37.6%, Avg loss: 2.149823 

Epoch 2
-------------------------------
loss: 2.160450  [   64/60000]
loss: 2.155108  [ 6464/60000]
loss: 2.097090  [12864/60000]
loss: 2.106639  [19264/60000]
loss: 2.060651  [25664/60000]
loss: 1.994428  [32064/60000]
loss: 2.023624  [38464/60000]
loss: 1.942107  [44864/60000]
loss: 1.941576  [51264/60000]
loss: 1.873218  [57664/60000]
Test Error: 
 Accuracy: 53.6%, Avg loss: 1.878297 

Epoch 3
-------------------------------
loss: 1.911321  [   64/60000]
loss: 1.887351  [ 6464/60000]
loss: 1.769241  [12864/60000]
loss: 1.803301  [19264/60000]
loss: 1.698864  [25664/60000]
loss: 1.645900  [32064/600